In [1]:
import torch
from torch import nn
import torch.nn.functional as F

from src.model import TimesFM

In [2]:
torch_model = TimesFM(context_len=64, horizon_len=24)

In [3]:
m = torch.load("./dev/timesfm_pytorch.pth")
torch_model.load_state_dict(m)

<All keys matched successfully>

In [4]:
from src.dataset import InferDataset
import pandas as pd

In [5]:
data = pd.read_csv("./data/ETTm1.csv")

data["ds"] = pd.to_datetime(data["date"])
data["unique_id"] = "AAA"

# delete the last 24 rows to make infer data
inferdata = data.iloc[:64]

dataset = InferDataset.from_dataframe(
    data=inferdata,
    freq="MIN",
    context_len=64,
    horizon_len=24,
    group_id="unique_id",
    target="OT",
    time_idx="ds",
)

In [6]:
x = dataset[0]['x'].unsqueeze(0)
paddings = dataset[0]['paddings'].unsqueeze(0)
freqs = dataset[0]['freq'].unsqueeze(0)

In [7]:
freqs

tensor([[0]], dtype=torch.int32)

In [8]:
fnal_out, full_out = torch_model.predict(x=x,paddings=paddings,freq=freqs)

In [9]:
fnal_out

tensor([[25.2414, 22.5359, 14.9787,  2.5792, 20.4304,  6.6938, 15.4990, 38.1884,
         33.0866, 25.9467, 21.7724, 19.4026, 25.6673, 29.9183, 35.6279, 27.6130,
         18.5815, 26.9171, 29.2396,  1.5767, 29.0011, 20.4224, 33.6068, 23.6053]])

In [11]:
target = data.iloc[64:(64+24)]["OT"].values
pred = fnal_out.squeeze().detach().numpy()